In [1]:
print("I am fine")

I am fine


In [2]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
from transformers import LlamaForCausalLM, LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained("/120040051/hf_llama2")
model = LlamaForCausalLM.from_pretrained("/120040051/hf_llama2").to(device)

/root/anaconda3/envs/llama2-py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]/root/anaconda3/envs/llama2-py311/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 3/3 [00:33<00:00, 11.18s/it]


In [5]:
def calculate_perplexity(sentence, tokenizer, model, device):
    model.eval()  # Ensure the model is in evaluation mode
    # Tokenize the input sentence
    encodings = tokenizer(sentence, return_tensors="pt")

    # Move encodings to the same device as the model
    input_ids = encodings.input_ids.to(device)
    target_ids = input_ids.clone()
    attention_mask = encodings.attention_mask.to(device)

    # Calculate Negative Log-Likelihood
    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids, attention_mask=attention_mask)
        neg_log_likelihood = outputs.loss * input_ids.size(1)  # Multiply by sequence length

    # Calculate Perplexity and append to list
    ppl = torch.exp(neg_log_likelihood / input_ids.size(1))  # Divide by sequence length
    perplexity = ppl.item()

    return perplexity

In [8]:
import json
with open("tmp_compositional_sents.json", 'r') as f:
    j_data = json.load(f)

sent_list = j_data["data"]
sent_list_ppl = []

i = 0
for sent_dict in sent_list:
    text = sent_dict["text"]
    ppl = calculate_perplexity(text, tokenizer, model, device)
    sent_dict["ppl"] = round(ppl, 2)
    sent_list_ppl.append(sent_dict)
    i += 1
    if (i % 100 == 0):
        print(f"Processed {i} sentences")
print(f"Processed {i} sentences")
j_data["data"] = sent_list_ppl

outfile = "sentence_table.json"
print(f"Writing the results to {outfile}, ...")
with open(outfile, 'w') as f:
    json.dump(j_data, f)
print("Writing completed")

Processed 100 sentences
Processed 200 sentences
Processed 300 sentences
Processed 400 sentences
Processed 500 sentences
Processed 600 sentences
Processed 700 sentences
Processed 800 sentences
Processed 900 sentences
Processed 1000 sentences
Processed 1100 sentences
Processed 1200 sentences
Processed 1300 sentences
Processed 1400 sentences
Processed 1500 sentences
Processed 1600 sentences
Processed 1600 sentences


In [6]:
sent1 = "A man is eating pizza in the restaurant."
perplexity_values = calculate_perplexity(sent1, tokenizer, model, device)
print(perplexity_values)

21.415712356567383
